In [1]:
import scqubits
import qutip
import qutip
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt

# Add drive and evolve two molecule,

### The coupling operator is $g (C_A^1 \hat{n}_A^1+C_A^2 \hat{n}_A^2)(C_B^1 \hat{n}_B^1+C_B^2 \hat{n}_B^2)$

# First, define the hillbert space of two molecules

In [2]:
EJ1 = 4
EJ2 = 3.5
EC = 1
EL = 1
molecule_coupling = 0.1
inter_molecule_coupling = 0.3
flux_ext = 0.5
g = 0.3
assymetry1 = 0.003
flux_1 = (2+assymetry1)/2*flux_ext
flux_2 = (2-assymetry1)/2*flux_ext
assymetry2 = 0.006
flux_3 = (2+assymetry2)/2*flux_ext
flux_4 = (2-assymetry2)/2*flux_ext
qubit_levels = 5

qubit1 = scqubits.Fluxonium(EJ = EJ1,
                    EC =EC,
                    EL = EL,
                    flux = flux_1,
                    cutoff = 110,
                    truncated_dim = qubit_levels)
qubit2 = scqubits.Fluxonium(EJ = EJ1,
                    EC =EC,
                    EL = EL,
                    flux = flux_2,
                    cutoff = 110,
                    truncated_dim = qubit_levels)

qubit3 = scqubits.Fluxonium(EJ = EJ2,
                    EC =EC,
                    EL = EL,
                    flux = flux_3,
                    cutoff = 110,
                    truncated_dim = qubit_levels)
qubit4 = scqubits.Fluxonium(EJ = EJ2,
                    EC =EC,
                    EL = EL,
                    flux = flux_4,
                    cutoff = 110,
                    truncated_dim = qubit_levels)


hilbertspace = scqubits.HilbertSpace([qubit1,qubit2,qubit3,qubit4])

hilbertspace.add_interaction(
    g_strength=molecule_coupling, op1=qubit1.phi_operator, op2=qubit2.phi_operator, add_hc=False
)
hilbertspace.add_interaction(
    g_strength=molecule_coupling, op1=qubit3.phi_operator, op2=qubit4.phi_operator, add_hc=False
)


CA1, CA2, CB1, CB2 = (0.5,0.5,0.5,0.5)
hilbertspace.add_interaction(
   qobj = inter_molecule_coupling * (
        CA1 * qutip.tensor(
        qutip.Qobj(qubit1.n_operator(energy_esys  = False)[:qubit_levels, :qubit_levels]),
        qutip.identity(dims = qubit2.truncated_dim),
        qutip.identity(dims = qubit3.truncated_dim),
        qutip.identity(dims = qubit4.truncated_dim))
        + CA2 * qutip.tensor(
        qutip.identity(dims = qubit1.truncated_dim),
        qutip.Qobj(qubit2.n_operator(energy_esys  = False)[:qubit_levels, :qubit_levels]),
        qutip.identity(dims = qubit3.truncated_dim),
        qutip.identity(dims = qubit4.truncated_dim))
   ) * (
    CB1 * qutip.tensor(
        qutip.identity(dims = qubit1.truncated_dim),
        qutip.identity(dims = qubit3.truncated_dim),
        qutip.Qobj(qubit3.n_operator(energy_esys  = False)[:qubit_levels, :qubit_levels]),
        qutip.identity(dims = qubit4.truncated_dim)
    )
    + CB2 *qutip.tensor(
        qutip.identity(dims = qubit1.truncated_dim),
        qutip.identity(dims = qubit3.truncated_dim),
        qutip.identity(dims = qubit4.truncated_dim),
        qutip.Qobj(qubit4.n_operator(energy_esys  = False)[:qubit_levels, :qubit_levels]),
    )
    
   )
)

In [4]:

def find_corresponding_dressed_eigenstate_index(state1, state2,H_coupled):
    eigenstates_coupled = H_coupled.eigenstates()[1]
    product_state = qutip.tensor(state1, state2)
    overlaps = [qutip.metrics.fidelity(eigenstate, product_state) for eigenstate in eigenstates_coupled]
    max_overlap = max(overlaps)
    most_overlapping_index = overlaps.index(max_overlap)
    return most_overlapping_index, max_overlap
